<a href="https://colab.research.google.com/github/Mogaveera/atmanomalydetection/blob/master/anomaly_6_classes_using_TransferLearningInceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

import pandas as pd
import numpy as np
import cv2
import os
import h5py
from tqdm import tqdm
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model, load_model, Sequential
from keras.layers import Input, LSTM, Dense, Dropout
from keras.layers import Bidirectional
from keras.utils import to_categorical
from keras.applications.imagenet_utils import preprocess_input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard,EarlyStopping
from keras.utils.io_utils import HDF5Matrix

SEQ_LEN = 30
BATCH_SIZE = 16
EPOCHS = 100

train_video_index = []
test_video_index = []

Using TensorFlow backend.


In [0]:
!git clone https://github.com/Mogaveera/anomalydetection.git

Cloning into 'anomalydetection'...
remote: Enumerating objects: 250, done.
remote: Total 250 (delta 0), reused 0 (delta 0), pack-reused 250
Receiving objects: 100% (250/250), 733.86 MiB | 49.04 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Checking out files: 100% (233/233), done.


In [0]:
def main():
    # Get model with pretrained weights.
    base_model = InceptionV3(
    weights='imagenet',
    include_top=True)
    
    
    # We'll extract features at the final pool layer.
    model = Model(
        inputs=base_model.input,
        outputs=base_model.get_layer('avg_pool').output)
    model.save('inception.h5')

    # Getting the data
    df = get_data('anomalydetection/Data/data_file.csv')
    
    # Clean the data
    # df_clean = clean_data(df)
    
    # Creating index-label maps and inverse_maps
    label_index, index_label = get_class_dict(df)
    
    # Split the dataset into train and test
    train, test = split_train_test(df)
    
    # Encoding the dataset
    train_video_index = make_dataset(train, model,label_index, "train")
    test_video_index = make_dataset(test, model,label_index,"test")
    return (train_video_index, test_video_index)

In [0]:
def get_data(path, if_pd=False):
    """Load our data from file."""
    names = ['partition', 'class', 'video_name']
    df = pd.read_csv(path,names=names)
    return df

def get_class_dict(df):
    class_name =  list(df['class'].unique())
    index = np.arange(0, len(class_name))
    label_index = dict(zip(class_name, index))
    index_label = dict(zip(index, class_name))
    return (label_index, index_label)
    
#def clean_data(df):
#    mask = np.logical_and(df['frames'] >= SEQ_LEN, df['frames'] <= MAX_SEQ_LEN)
#    df = df[mask]`
#    return df

def split_train_test(df):
    partition =  (df.groupby(['partition']))
    un = df['partition'].unique()
    train = partition.get_group(un[0])
    test = partition.get_group(un[1])
    return (train, test)

def preprocess_image(img):
    img = cv2.resize(img, (299,299))
    return preprocess_input(img)
    
    
def video_to_frame(row, model,label_index, phase, not_created):
    input_f = []
    output_y = []
    index = 0
    cap = cv2.VideoCapture(os.path.join("anomalydetection/Data","anomaly_dataset",str(row["class"].iloc[0]) ,str(row["video_name"].iloc[0]) + ".mp4")) 
    #print(str(row["class"].iloc[0]))
    #print(str(row["video_name"].iloc[0]))
    frameno = 1
    imgs = []
    length = 0
    seq = 12
    while (cap.isOpened()):
      ret, frame = cap.read()
      if ret:
        if length < seq:
          if frameno % 10 == 0:
            frameno = frameno + 1
            frame = preprocess_image(frame)
            frame = image.img_to_array(frame)
            frame = frame / 255
            # features = model.predict(frame)
            imgs.append(frame)
            length = length + 1
          else:
            frameno = frameno + 1
        else:
          seq = seq + 12
          imgs1 = np.array(imgs)
          features = model.predict(imgs1)
          input_f.append(features)
          output_y.append(label_index)
          del imgs[:]
      else:
        break

    if not_created:
      f = h5py.File(phase+'_4'+'.h5', 'w')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        f.create_dataset(phase, data=input_f1, maxshape=(None, 12, 2048))
        f.create_dataset(phase+"_labels", data=output_y1, maxshape=(None,))
        f.close()
    else:
      hf = h5py.File(phase+'_4'+'.h5', 'a')
      input_f1 = np.array(input_f)
      output_y1 = np.array(output_y)
      index = input_f1.shape[0]
      if index > 0:
        hf[phase].resize((hf[phase].shape[0] + input_f1.shape[0]), axis = 0)
        hf[phase][-input_f1.shape[0]:] = input_f1

        hf[phase+"_labels"].resize((hf[phase+"_labels"].shape[0] + output_y1.shape[0]), axis = 0)
        hf[phase+"_labels"][-output_y1.shape[0]:] = output_y1
        hf.close()

    del input_f[:]
    del output_y[:]
    del imgs[:]
    cap.release()
    return index
    
    


def make_dataset(data, model,label_index, phase):
    video_index = [0]
    required_classes = ["Abuse", "Arson","Assault", "Burglary", "Fighting", "normal"]
   
    not_created = True
    for i in tqdm(range(data.shape[0])):
    # Check whether the given row , is of a class that is required
        if str(data.iloc[[i]]["class"].iloc[0]) in required_classes:
            index = required_classes.index(str(data.iloc[[i]]["class"].iloc[0]))
            label_index = index
            index = video_to_frame(data.iloc[[i]], model,label_index, phase, not_created)
            real_index = video_index[-1] + index
            video_index.append(real_index)
            if real_index > 0:
              not_created = False

    return video_index

In [0]:
train_video_index, test_video_index = main()














96116736/96112376 [==============================] - 1s 0us/step


100%|██████████| 39/39 [00:45<00:00,  1.73s/it]


In [0]:
x_train = HDF5Matrix('train_4.h5', 'train')
y_train = HDF5Matrix('train_4.h5', 'train_labels')
x_test = HDF5Matrix('test_4.h5', 'test')
y_test = HDF5Matrix('test_4.h5', 'test_labels')

print(x_train.shape)
print(y_train.shape)
print(y_train[783])
print(x_test.shape)
print(y_test.shape)
print(y_test[93])
print(train_video_index)
print(test_video_index)

(1496, 12, 2048)
(1496,)
2
(378, 12, 2048)
(378,)
0
[]
[]


In [0]:
y_test1 = np.array(y_test)
print(y_test1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5]


In [0]:
def lstm():
    """Build a simple LSTM network. We pass the extracted features from
    our CNN to this model predominantly."""
    input_shape = (12, 2048)
    # Model.
    model = Sequential()
    model.add(Bidirectional(LSTM(2048), input_shape=input_shape))
    model.add(Dropout(0.8))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.8))
    model.add(Dense(6, activation='softmax'))
    checkpoint = ModelCheckpoint(filepath='models\\checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
    
    tb_callback = TensorBoard(
    log_dir="logs",
    histogram_freq=2,
    write_graph=True
    )
    
    callback_list = [checkpoint]
    
    optimizer = Adam(lr=1e-5, decay=1e-6)
    metrics = ['accuracy', 'top_k_categorical_accuracy']
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    #return model, callback_list
    #model.compile(optimizer = tf.train.AdamOptimizer(),
    #          loss = 'categorical_crossentropy',
    #        metrics=['accuracy'])
    return model

In [0]:
lstm = lstm()
lstm.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 4096)              67125248  
_________________________________________________________________
dropout_1 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               2097664   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 3078      
Total params: 69,225,990
Trainable params: 69,225,990
Non-trainable params: 0
_____________________________________________________

In [0]:
lstm.fit(x_train, y_train, batch_size = 16, epochs = 20,verbose = 2, validation_data = (x_test, y_test), shuffle = 'batch')
lstm.save("Anomaly_Recognition_4classes.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 1496 samples, validate on 378 samples
Epoch 1/20





 - 15s - loss: 2.3916 - acc: 0.1791 - val_loss: 1.6199 - val_acc: 0.2857
Epoch 2/20
 - 9s - loss: 1.9911 - acc: 0.2487 - val_loss: 1.5778 - val_acc: 0.2910
Epoch 3/20
 - 9s - loss: 1.8042 - acc: 0.2941 - val_loss: 1.5750 - val_acc: 0.3254
Epoch 4/20
 - 9s - loss: 1.6886 - acc: 0.3376 - val_loss: 1.5370 - val_acc: 0.3228
Epoch 5/20
 - 10s - loss: 1.5780 - acc: 0.3763 - val_loss: 1.4705 - val_acc: 0.4206
Epoch 6/20
 - 9s - loss: 1.4898 - acc: 0.4198 - val_loss: 1.3996 - val_acc: 0.4630
Epoch 7/20
 - 9s - loss: 1.4475 - acc: 0.4445 - val_loss: 1.3226 - val_acc: 0.6349
Epoch 8/20
 - 9s - loss: 1.3149 - acc: 0.5040 - val_loss: 1.2418 - val_acc: 0.6402
Epoch 9/20
 - 9s - loss: 1.2820 - acc: 0.5227 - val_loss: 1.3004 - val_acc: 0.4974
Epoch 10/20
 - 9s - loss: 1.1765 - acc: 0.5648 - val_loss: 1.1575 - val_acc: 0.6455
Epoch 11/20
 - 9s 

In [0]:
test_loss, test_acc = lstm.evaluate(x_test, y_test, batch_size=16)
print("accuracy: ", test_acc)

378/378 [==============================] - 1s 2ms/step
accuracy:  0.6402116398962717
